In [1]:
!pip install -q flwr tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 44.0.3 which is incompatible.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.3 which is incompatible.


In [2]:
%%writefile server.py
import flwr as fl

if __name__ == "__main__":
    # Define the strategy: we want to wait for at least 2 clients
    strategy = fl.server.strategy.FedAvg(
        min_fit_clients=2,        # Minimum number of clients to be sampled for the next round
        min_available_clients=2,  # Minimum number of clients that need to be connected to start
    )

    # Start the server
    print("Server starting... Waiting for 2 clients to connect.")
    fl.server.start_server(
        server_address="0.0.0.0:8080",
        config=fl.server.ServerConfig(num_rounds=3),
        strategy=strategy
    )

Writing server.py


In [3]:
%%writefile client.py
import flwr as fl
import tensorflow as tf
import sys
import numpy as np

# 1. Parse Client ID from command line (to simulate different data partitions)
# Example usage: python client.py 0  OR  python client.py 1
if len(sys.argv) > 1:
    client_id = int(sys.argv[1])
else:
    client_id = 0

# 2. Data Pre-processing and Partitioning
# We load the full dataset, but we slice it based on client_id
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Simple logic to split data into 2 partitions
split_idx = len(x_train) // 2
if client_id == 0:
    # Client 0 gets the first half
    x_train_local = x_train[:split_idx]
    y_train_local = y_train[:split_idx]
    print(f"Client {client_id}: Using data indices 0 to {split_idx}")
else:
    # Client 1 gets the second half
    x_train_local = x_train[split_idx:]
    y_train_local = y_train[split_idx:]
    print(f"Client {client_id}: Using data indices {split_idx} to {len(x_train)}")

# 3. Define the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 4. Define Flower Client
class MyClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        return model.get_weights()

    def fit(self, parameters, config):
        model.set_weights(parameters)
        # Train on the LOCAL partition
        model.fit(x_train_local, y_train_local, epochs=1, batch_size=32, verbose=0)
        return model.get_weights(), len(x_train_local), {}

    def evaluate(self, parameters, config):
        model.set_weights(parameters)
        loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
        return loss, len(x_test), {"accuracy": accuracy}

# 5. Connect to Server
print(f"Client {client_id} connecting to server...")
fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=MyClient())

Writing client.py


In [4]:
import subprocess
import time

# 1. Start Server in the background
print("🚀 Starting Server in background...")
server_process = subprocess.Popen(["python", "server.py"])
time.sleep(3) # Wait for server to initialize

# 2. Start Client 0 in the background (Uses first half of data)
print("👤 Starting Client 0 in background...")
client0_process = subprocess.Popen(["python", "client.py", "0"])

# 3. Start Client 1 in the foreground (Uses second half of data)
# We run this with ! so we can see the output logs in real-time
print("👤 Starting Client 1 in foreground...")
!python client.py 1

# 4. Cleanup: Kill processes after training is done
print("\n✅ Training Complete. Shutting down background processes.")
server_process.kill()
client0_process.kill()

🚀 Starting Server in background...
👤 Starting Client 0 in background...
👤 Starting Client 1 in foreground...
2026-01-21 08:56:23.744014: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-21 08:56:23.750218: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-21 08:56:23.774158: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768985783.815358     935 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768985783.827191     935 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768985783.855707   